# Assignment_3 Answers

In [3]:

## Amazon Search Answer

import selenium
import pandas as pd
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

driver = webdriver.Chrome(r"F:\webdriver/chromedriver.exe")
driver.get('https://www.amazon.in')

search_box = driver.find_element_by_id('twotabsearchtextbox')
search_box.send_keys("guiter")

search_button = driver.find_element_by_id("nav-search-submit-text").click()

In [4]:
## amazon.in Answer

from autoscraper import AutoScraper

url='https://www.amazon.in/s?k=guiter&ref=nb_sb_noss'

wanted_list=['₹2,199 ','Intern INT-38C Acoustic Guitar Kit, With Bag, Strings, Pick And Strap, Black','13,329']

scraper=AutoScraper()
result=scraper.build(url, wanted_list)
print(result)

scraper.get_result_similar(url,grouped=True)

scraper.set_rule_aliases({'rule_92hh':'Title', 'rule_odpb':'Rating'})
scraper.keep_rules(['rule_92hh','rule_odpb'])
scraper.save('amazon.search')

results=scraper.get_result_similar('https://www.amazon.in/s?k=guiter&ref=nb_sb_noss', group_by_alias=True)


### errors are repeating while taking 'rule'


['₹2,199', '₹6,990', '₹1,790', '₹6,495', '₹3,330', '₹6,500', '₹2,895', '₹1,899', '₹1,999', '₹2,399', '₹2,499', '₹7,490', '₹2,599', '₹3,499', '₹8,990', '₹2,859', '₹4,999', '₹2,016', 'Intern INT-38C Acoustic Guitar Kit, With Bag, Strings, Pick And Strap, Black', 'Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 38C with Bag, Strings, Pick and Strap, TBS Transparent Blue Sunburst', 'Juarez Acoustic Guitar, 38 Inch Cutaway with Pick Guard, 38CPG with Bag, Strings, Pick and Strap, Black', 'Juârez Acoustic Guitar, 38 Inch Cutaway, JRZ38C with Bag, Strings, Pick and Strap, Red', 'JUAREZ JRZ38C Right Handed Acoustic Guitar with Bag, Strings, Pick and Strap (Natural, 6 Strings)', 'Juarez Acoustic Guitar, 38 Inch Cutaway, 38CMB with Bag, Strings, Pick and Strap, Matte Black', 'Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 38C with Bag, Strings, Pick and Strap, 3TS Sunburst', 'JUAREZ Arpéggio 41 Inch Acoustic Guitar Kit, Spruce Top, Black', 'Juârez JRZ38C/MAH 6 Strings Acoustic Guitar 38 Inch Cutaw

KeyError: 'rule_92hh'

In [5]:

### Google.Image.com Answer

import os
import time

import io
import hashlib
import signal
from glob import glob
import requests

from PIL import Image
from selenium import webdriver

number_of_images = 400
GET_IMAGE_TIMEOUT = 2
SLEEP_BETWEEN_INTERACTIONS = 0.1
SLEEP_BEFORE_MORE = 5
IMAGE_QUALITY = 85

output_path = "/path/to/your/image/directory"

search_terms = [
    "fruits icon",
    "cars icon",
    "Meachine Learning",
   
]

dirs = glob(output_path + "*")
dirs = [dir.split("/")[-1].replace("_", " ") for dir in dirs]
search_terms = [term for term in search_terms if term not in dirs]

driver = webdriver.Chrome(r"F:\webdriver/chromedriver.exe")
driver.get("https://google.com")

class timeout:
    def __init__(self, seconds=1, error_message="Timeout"):
        self.seconds = seconds
        self.error_message = error_message

    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)

    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)

    def __exit__(self, type, value, traceback):
        signal.alarm(0)

def fetch_image_urls(
    query: str,
    max_links_to_fetch: int,
    wd: webdriver,
    sleep_between_interactions: int = 1,
):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # Build the Google Query.
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    # Declared as a set, to prevent duplicates.
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # Get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(
            f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}"
        )

        # Loop through image thumbnail identified
        for img in thumbnail_results[results_start:number_results]:
            # Try to click every thumbnail such that we can get the real image behind it.
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # Extract image urls
            actual_images = wd.find_elements_by_css_selector("img.n3VNCb")
            for actual_image in actual_images:
                if actual_image.get_attribute(
                    "src"
                ) and "http" in actual_image.get_attribute("src"):
                    image_urls.add(actual_image.get_attribute("src"))

            image_count = len(image_urls)

            # If the number images found exceeds our `num_of_images`, end the seaerch.
            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            # If we haven't found all the images we want, let's look for more.
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(SLEEP_BEFORE_MORE)

            # Check for button signifying no more images.
            not_what_you_want_button = ""
            try:
                not_what_you_want_button = wd.find_element_by_css_selector(".r0zKGf")
            except:
                pass

            # If there are no more images return.
            if not_what_you_want_button:
                print("No more images available.")
                return image_urls

            # If there is a "Load More" button, click it.
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button and not not_what_you_want_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # Move the result startpoint further down.
        results_start = len(thumbnail_results)

    return image_urls


def persist_image(folder_path: str, url: str):
    try:
        print("Getting image")
        # Download the image.  If timeout is exceeded, throw an error.
        with timeout(GET_IMAGE_TIMEOUT):
            image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        # Convert the image into a bit stream, then save it.
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert("RGB")
        # Create a unique filepath from the contents of the image.
        file_path = os.path.join(
            folder_path, hashlib.sha1(image_content).hexdigest()[:10] + ".jpg"
        )
        with open(file_path, "wb") as f:
            image.save(f, "JPEG", quality=IMAGE_QUALITY)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

def search_and_download(search_term: str, target_path="./images", number_images=5):
    # Create a folder name.
    target_folder = os.path.join(target_path, "_".join(search_term.lower().split(" ")))

    # Create image folder if needed.
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Open Chrome
    with webdriver.Chrome() as wd:
        # Search for images URLs.
        res = fetch_image_urls(
            search_term,
            number_images,
            wd=wd,
            sleep_between_interactions=SLEEP_BETWEEN_INTERACTIONS,
        )

        # Download the images.
        if res is not None:
            for elem in res:
                persist_image(target_folder, elem)
        else:
            print(f"Failed to return links for term: {search_term}")

# Loop through all the search terms.
for term in search_terms:
    search_and_download(term, output_path, number_of_images)



Found: 100 search results. Extracting links from 0:100


KeyboardInterrupt: 

In [6]:

#### Flipkart Smartphone Answer

from autoscraper import AutoScraper

flipkart_url='https://www.flipkart.com/search?q=mobiles'
wanted_list=['realme C21Y (Cross Blue, 64 GB)', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '13MP + 2MP + 2MP | 5MP Front Camera', 
            '1,12,238 Ratings', '1 Year Warranty for Mobile and 6 Months for Accessories', '5000 mAh Battery', 'Unisoc T610 Processor']

scraper=AutoScraper()
result=scraper.build(flipkart_url,wanted_list)
print(result)

scraper.get_result_similar(flipkart_url,grouped=True)

Name=[]
RAM_ROM_Storages=[]
Camera=[]
waranty=[]
Rating=[]
Battery=[]
Processor=[]

scraper.set_rule_aliases({'rule_6jfj':Name, 'rule_zepu':RAM_ROM_Storages, 'rule_l1mo':Camera, 'rule_n2mp':waranty,
                         'rule_nreb':Rating, 'rule_obhd':Battery, 'rule_as3m':Processor})
scraper.keep_rules(['rule_6jfj','rule_zepu', 'rule_l1mo', 'rule_n2mp', 'rule_nreb', 'rule_obhd', 'rule_as3m'])
scraper.save('flipkart.search')

### after execution key errors are getting even after through checkings.

['realme C21Y (Cross Blue, 64 GB)', 'realme C21Y (Cross Black, 64 GB)', 'realme C21Y (Cross Blue, 32 GB)', 'SAMSUNG Galaxy F22 (Denim Black, 128 GB)', 'realme C11 2021 (Cool Blue, 32 GB)', 'realme C11 2021 (Cool Grey, 32 GB)', 'POCO M3 (Cool Blue, 64 GB)', 'SAMSUNG Galaxy F22 (Denim Black, 64 GB)', 'realme C11 2021 (Cool Grey, 64 GB)', 'realme Narzo 50A (Oxygen Blue, 64 GB)', 'realme Narzo 50A (Oxygen Blue, 128 GB)', 'Nokia C01 Plus (Grey, 16 GB)', 'realme Narzo 50A (Oxygen Green, 64 GB)', 'realme Narzo 50A (Oxygen Green, 128 GB)', 'realme C11 2021 (Cool Blue, 64 GB)', 'SAMSUNG Galaxy F42 5G (Matte Aqua, 128 GB)', 'POCO C3 (Arctic Blue, 32 GB)', 'POCO X3 Pro (Steel Blue, 128 GB)', 'REDMI 9i Sport (Carbon Black, 64 GB)', 'SAMSUNG Galaxy F22 (Denim Blue, 128 GB)', 'REDMI 9i Sport (Coral Green, 64 GB)', 'Infinix Hot 10 Play (Obsidian Black, 32 GB)', 'realme C25Y (Glacier Blue, 128 GB)', 'SAMSUNG Galaxy A52s 5G (Awesome White, 128 GB)', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '3 G

KeyError: 'rule_6jfj'

In [8]:
## GeoMapping Answer

import pandas as pd
from geopy.geocoders import Nominatim

#### the csv file had downloaded from net

df = pd.read_csv("https://raw.githubusercontent.com/tiffsea/python-code/master/demos/data.csv",
                 sep=',') 

# print the data
df.head()

#create variable column inside file called `myAddress and assign it to seleted address columns
df['query'] = df['COMPANY'] + " " + df['CITY'] + " " +  df['COUNTRY']

#print new column with index -first 5 rows only
df.iloc[0:5, 8:9]

#remove duplicate addresses (new concat column) but keep first instance
df.drop_duplicates(subset ='query', keep ='first', inplace = True)

#print some useful info: row length and shape size
print("data row x columns is {}\ndata row count is {}".format(df.shape,len(df.index)))

#print first rows as sample
df.head()

#drop columns we won't use
df = df.drop(columns=['DISTRICT','COUNTY'])
                 
#print row length and shape size
print("data row x columns is {}\ndata row count is {}".format(df.shape,len(df.index)))

#print first rows as sample
df.head()

#create 2 new columns to store lat/long - initalise to null
df['location_lat'] = ""
df['location_long'] = ""
df['location_address'] = ""

#print first rows to sample
df.head()

geolocator = Nominatim(user_agent="myApp")

for i in df.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(df['query'][i])
        
        #append lat/long to column using dataframe location
        df.loc[i,'location_lat'] = location.latitude
        df.loc[i,'location_long'] = location.longitude
        df.loc[i,'location_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        df.loc[i,'location_lat'] = ""
        df.loc[i,'location_long'] = ""
        df.loc[i,'location_address'] = ""

#print first rows as sample
df.head()


#write the contents thus far to new csv file
###df.to_csv('geopy_data.csv')



data row x columns is (1, 9)
data row count is 1
data row x columns is (1, 7)
data row count is 1


,COMPANY,STREET_ADDRESS,CITY,STATE,COUNTRY,POSTCODE,query,location_lat,location_long,location_address
0,University of Sussex,NaN,Brighton,NaN,UK,NaN,University of Sussex Brighton UK,50.868,-0.0877856,"University of Sussex, University of Sussex Bou..."


In [9]:
## Trak.in ANswer


from bs4 import BeautifulSoup
import requests

url= 'https://trak.in/india-startup-funding-investment-2015/'

response = requests.get(url)
response.status_code

soup = BeautifulSoup(response.content, 'html.parser')

print(soup.prettify)

more_data_urls = [url]     

for h2_tag in soup.find_all(name="h2"):
    more_data_urls.append(h3_tag.find(name='a').get('href'))

more_data_urls[:]

new_row_list = []
column_name = ['Sr. No.', 'Date (dd/mm/yyyy)', 'Startup Name', 'Industry/ Vertical', 'Sub-Vertical', 'City / Location', 'Investors’ Name', 'Investment Type', 'Amount (in USD)']
more_data_urls = set(more_data_urls)

urls_count = 1
for url in more_data_urls:
    html_response = requests.get(url)
    html_response.status_code
    soup = BeautifulSoup(html_response.content, 'html.parser')
    
    class_list = []
    for element in soup.find_all(class_=True):
        class_list.extend(element["class"])
    class_list = [cls for cls in class_list if 'tablepress-id-' in cls] 
    
    if len(class_list) < 1:
        skip_first_row = True
        class_list.append(None)
        for class_ in class_list:
            tbl=soup.find(name='table') #, class_=class_)

            n_rows = 0
            for tr in tbl.find_all('tr'):
                if skip_first_row == True:
                    skip_first_row = False
                    continue
                new_row = {}
                for col_id, td in enumerate(tr.find_all('td')):
                    if col_id < len(column_name):
                        new_row[column_name[col_id]] = td.text
                if not new_row == {}:
                    n_rows += 1
                    new_row_list.append(new_row)
            #print("class_list-old:", class_, len(new_row_list), n_rows, url)
    else:
        for class_ in class_list:
            tbl=soup.find(name='table', class_=class_)

            n_rows = 0
            for tr in tbl.find_all('tr'):
                new_row = {}
                for col_id, td in enumerate(tr.find_all('td')):
                    if col_id < len(column_name):
                        new_row[column_name[col_id]] = td.text
                if not new_row == {}:
                    n_rows += 1
                    new_row_list.append(new_row)
            #print("class_list-new :", class_, len(new_row_list), n_rows, url)

data = pd.DataFrame(new_row_list, columns=column_name)
print("Data shape :", data.shape)

data

### wheteever giving h2 or h3  but it showing from 2015 to 2018 even taking source from 2021 or 2020

<bound method Tag.prettify of  <!DOCTYPE html>

<!--[if IE 8]>
	<html class="ie ie8" lang="en-US"> <![endif]-->
<!--[if IE 9]>
	<html class="ie ie9" lang="en-US"> <![endif]-->
<!--[if gt IE 9]><!-->
<html lang="en-US"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://trak.in/xmlrpc.php" rel="pingback"/>
<title>Indian Startup Funding And Investment Chart [2021] – Trak.in – Indian Business of Tech, Mobile &amp; Startups</title>
<link href="https://trak.in/india-startup-funding-investment-2015/amp/" rel="amphtml"/><meta content="AMP for WP 1.0.64.1" name="generator">
<meta content="en_us" property="og:locale">
<meta content="Trak.in - Indian Business of Tech, Mobile &amp; Startups" property="og:site_name">
<meta content="https://trak.in/india-startup-funding-investment-2015/" property="og:url"/>
<meta content="Indian Startup Funding And Investme

NameError: name 'h3_tag' is not defined

In [10]:
## digi.in Answer


from autoscraper import AutoScraper

digi='https://www.digit.in/top-products/best-budget-gaming-laptop-3591.html'
wanted_list = ['ACER ASPIRE 7 A715-75G-50SA', 'WINDOWS 10', '15.6" (1920 X 1080)', 
               'CORE I5 9TH GEN | 2.4 GHZ WITH TURBO BOOST UPTO 4.1 GHZ', '512 NA/8 GBGB DDR4']

scraper=AutoScraper()
result=scraper.build(digi, wanted_list)
print(result)

scraper.get_result_similar(digi, grouped=True)

import pandas as pd

Model_Type=[]
Os=[]
Display=[]
Processor=[]
Memory=[]


scraper.set_rule_aliases({'rule_jl3j':Model_Type, 'rule_bw7ai':Os})
scraper.keep_rules(['rule_jl3j', 'rule_bw7ai'])            

##### every time the rules are chaniging when going for run and shoiwng this error
                                                               
scraper.save('digi.search')

results=scraper.get_result_similar('https://www.digit.in/top-products/best-gaming-laptops-40.html',group_by_alias=True)



['ACER ASPIRE 7 A715-75G-50SA', 'MSI GF63 THIN 9SC-240IN', 'HP PAVILLION 15-BC512TX', 'ACER NITRO 5 AN517-51-53JG', 'LENOVO LEGION Y7000 81V4000LIN', 'ASUS TUF GAMING FX505DD-AL185T', 'HP PAVILION GAMING 15-EC0026AX', 'ASUS TUF GAMING FX505DY-BQ024T', 'MI NOTEBOOK HORIZON EDITION 14', 'ASUS VIVOBOOK 14 X412FJ-EK513T', 'Windows 10', '15.6" (1920 x 1080)', 'Core i5 9th Gen | 2.4 GHz with Turbo Boost Upto 4.1 GHz', '512 NA/8 GBGB DDR4', '2.15', '363.4 x 254.5 x 23.25', 'NVIDIA Geforce GTX 1650', '512 GB NA/8 GBGB DDR4', '1.86', '359 x 254 x 21.7', 'NVIDIA Geforce GTX 1650 Max Q', '9th Gen Intel Core i5  | NA', '2.17', '36.5 x 25.7 x 2.5', 'NVIDIA GeForce GTX 1050', '17.3" (1920 x 1080)', 'Core i5 9th Gen | 2.4 GHz', '1 TB NA/8 GBGB DDR4', '2.7', '403 x 280 x 26.9', 'NVIDIA Geforce GTX 1050', '9th gen Intel Core i5 | 2.4 Ghz', '1 GB NA/8 GBGB DDR4', '2.3', '36 x 26.7 x 2.6', 'NVIDIA GEFORCE GTX 1050 (3GB GDDR5)', 'NA', '2.2', '26.2 x 36 x 2.6', '1.98', '25.7 x 36 x 2.4', 'AMD Ryzen 5-3550H

KeyError: 'rule_jl3j'

In [11]:
### Forbes.com Answer
import numpy as np
import pandas as pd


forbes = pd.read_csv('https://raw.githubusercontent.com/enlitedata/repos/main/Forbes_static.csv',
                              names = ['Rank', 'Name', 'Net Worth', 'Age', 'Source', 'Country'])

forbes_realtime = pd.read_csv('https://raw.githubusercontent.com/enlitedata/repos/main/Forbes_all.csv',
                              names = ['Rank', 'Name', 'Net Worth', 'Age', 'Source', 'Country'])

forbes_women = pd.read_csv('https://raw.githubusercontent.com/enlitedata/repos/main/women_forbes.csv',
                             names = ['Rank', 'Name', 'Net Worth', 'Age', 'Source', 'Country'])


#creating data frame for current net worth
realtime = forbes_realtime[['Name','Net Worth']]

#merging dataframe
forbes_data = pd.merge(forbes, realtime,how = 'left', on = 'Name')

#cleaning NA values
forbes_data['Age'] = forbes_data['Age'].fillna(forbes_data['Age'].median())
forbes_data['Net Worth_y'] = forbes_data['Net Worth_y'].fillna(0)

#creating new column "Change"
forbes_data['Change'] = forbes_data['Net Worth_y']- forbes_data['Net Worth_x']


# changing float into integar
forbes_data['Age'] = list(map(lambda x: int(x), forbes_data['Age']))
forbes_data['Rank'] = list(map(lambda x: int(x), forbes_data['Rank']))


#creating women data frame, so i can merge that into main data frame to create Gender column
women = forbes_women['Name']

#Creating new column "Gender" in main data frame
forbes_data['Gender'] = forbes_data['Name'].isin(women)
forbes_data['Gender'].replace([True,False],['F','M'],inplace = True)

#checking final data frame
forbes_data



,Rank,Name,Net Worth_x,Age,Source,Country,Net Worth_y,Change,Gender
0,1,Jeff Bezos,112.0,54,Amazon,United States,147.3,35.3,M
1,2,Bill Gates,90.0,62,Microsoft,United States,95.9,5.9,M
2,3,Warren Buffett,84.0,88,Berkshire Hathaway,United States,85.1,1.1,M
3,4,Bernard Arnault,72.0,69,LVMH,France,71.2,-0.8,M
4,5,Mark Zuckerberg,71.0,34,Facebook,United States,57.6,-13.4,M
...,...,...,...,...,...,...,...,...,...
2207,2124,Zhao Xiaoqiang,1.0,51,fashion entertainment,China,0.0,-1.0,M
2208,2124,Zhou Liangzhang,1.0,55,electrical equipment,China,0.0,-1.0,M
2209,2124,Zhu Xingming,1.0,51,electrical equipment,China,0.0,-1.0,M
2210,2124,Zhuo Jun,1.0,52,printed circuit boards,Hong Kong,1.1,0.1,M
